In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "gemma2-9b-it", temperature = 0 )

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system"," You are a helpful assistant. You will provide answers in below: {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [7]:
from langchain_core.messages import HumanMessage, AIMessage

In [9]:
chain.invoke({"messages": [HumanMessage(content = "My name is Kunal Patil")]})

AIMessage(content="Hello Kunal Patil!\n\nIt's nice to meet you.  \n\nI'm ready to assist you with any questions or tasks you may have. Just ask!  \n\nWhat can I do for you today? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 31, 'total_tokens': 82, 'completion_time': 0.092727273, 'prompt_time': 0.001483969, 'queue_time': 0.091930733, 'total_time': 0.094211242}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d624d22d-4697-40d9-bb1c-17ccbc1c5d4e-0', usage_metadata={'input_tokens': 31, 'output_tokens': 51, 'total_tokens': 82})

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [12]:
config = {"configurable":{"session_id": "chat3"}}

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="My name is Kunal")], 
    config=config
)

In [14]:
response.content

"Hello Kunal, it's nice to meet you!\n\nI'm ready to assist you with any questions or tasks you may have. Just ask, and I'll do my best to help.  \n\nWhat can I do for you today? 😊  \n\n"

In [16]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [17]:
config = {"configurable": {"session_id":"chat4"}}

In [19]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content = "My name is Kunal Patil and I am Learning Agentic AI")],
        "language": "Hindi"
    }, config = config
)

In [20]:
response.content

'नमस्ते कुणाल पटिल! \n\nयह बहुत अच्छा है कि आप एजेंटिक एआई सीख रहे हैं। यह एक रोमांचक क्षेत्र है! \n\nक्या आप एजेंटिक एआई के बारे में कुछ विशिष्ट सवाल पूछना चाहेंगे? \n\nमैं आपकी मदद करने के लिए यहाँ हूँ। \n\n'

In [32]:
## Manging session history 

from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 70,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

In [40]:
messages = [
    HumanMessage(content = "Hi, my name is Kunal"),
    SystemMessage(content = "Hi"),
    HumanMessage(content= "Give me some Ice cream Flavours"),
    SystemMessage(content = "Vanilla Chocolate Mango"),
    HumanMessage(content="What is 2 + 2?"),
    SystemMessage(content="It is 4")
]

In [41]:
trimmer.invoke(messages)

ImportError: Could not import transformers python package. This is needed in order to calculate get_token_ids. Please install it with `pip install transformers`.